In [5]:
# save in a format readable by c
import numpy as np
import torch

data = torch.load('pytorch_model.bin', map_location='cpu')

In [4]:
for i in data.keys():
    print(i, data[i].shape)

wte.weight torch.Size([50257, 768])
wpe.weight torch.Size([1024, 768])
h.0.ln_1.weight torch.Size([768])
h.0.ln_1.bias torch.Size([768])
h.0.attn.bias torch.Size([1, 1, 1024, 1024])
h.0.attn.c_attn.weight torch.Size([768, 2304])
h.0.attn.c_attn.bias torch.Size([2304])
h.0.attn.c_proj.weight torch.Size([768, 768])
h.0.attn.c_proj.bias torch.Size([768])
h.0.ln_2.weight torch.Size([768])
h.0.ln_2.bias torch.Size([768])
h.0.mlp.c_fc.weight torch.Size([768, 3072])
h.0.mlp.c_fc.bias torch.Size([3072])
h.0.mlp.c_proj.weight torch.Size([3072, 768])
h.0.mlp.c_proj.bias torch.Size([768])
h.1.ln_1.weight torch.Size([768])
h.1.ln_1.bias torch.Size([768])
h.1.attn.bias torch.Size([1, 1, 1024, 1024])
h.1.attn.c_attn.weight torch.Size([768, 2304])
h.1.attn.c_attn.bias torch.Size([2304])
h.1.attn.c_proj.weight torch.Size([768, 768])
h.1.attn.c_proj.bias torch.Size([768])
h.1.ln_2.weight torch.Size([768])
h.1.ln_2.bias torch.Size([768])
h.1.mlp.c_fc.weight torch.Size([768, 3072])
h.1.mlp.c_fc.bias torc

In [15]:
data['h.0.attn.c_attn.bias'].shape

torch.Size([2304])

In [3]:
data['h.0.attn.c_proj.weight'].shape

torch.Size([768, 768])

In [6]:
import os
os.makedirs("gpt_weights", exist_ok=True)
for key in data.keys():
    if "attn.c_attn" in key:
        # split to q, k, v
        q, k, v = np.split(data[key].numpy(), 3, axis=-1)
        print(q.shape, k.shape, v.shape, key.shape)
        # q.tofile(f"gpt_weights/{key}.q.bin")
        # k.tofile(f"gpt_weights/{key}.k.bin")
        # v.tofile(f"gpt_weights/{key}.v.bin")
    else:
        pass
        # data[key].numpy().tofile(f"gpt_weights/{key}.bin")
        # print(data[key].numpy().shape)

# save transpose of wte.weight as etw.weight

etw = data['wte.weight'].numpy().T
etw.tofile("gpt_weights/etw.weight.bin")

(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)
(768, 768) (768, 768) (768, 768)
(768,) (768,) (768,)


In [2]:
model.config._attn_implementation = 'eager'

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.42.4",
  "use_cache": true,
  "vocab_size": 50257
}

In [1]:
from transformers import GPT2Tokenizer, GPT2Model
# import model for text generation
from transformers import AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')

text = "Hello, my name is"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input['input_ids'].shape)
output = model(**encoded_input, output_hidden_states=True)
tokenizer.decode(output.logits.argmax(2)[0][-1])

torch.Size([1, 5])
tensor([[[-0.0875, -0.3301,  0.0152,  ...,  0.0285, -0.0014,  0.0068],
         [ 0.0355, -0.0567, -0.0626,  ...,  0.0619, -0.0195, -0.0601],
         [ 0.1620,  0.0243,  0.1283,  ...,  0.0703,  0.1250,  0.0067],
         [-0.2240,  0.0899,  0.2419,  ...,  0.0529, -0.0414, -0.1127],
         [-0.0021, -0.0150,  0.1825,  ...,  0.1230, -0.0282, -0.0324]]],
       grad_fn=<AddBackward0>)
torch.Size([1, 5, 768])
attn_output tensor([[[ 0.2490, -0.4001, -0.2390,  ...,  0.0205, -0.0261,  0.1027],
         [ 0.0972, -0.2530, -0.3704,  ..., -0.0196,  0.0435,  0.0473],
         [-0.3973,  0.1172, -0.0790,  ..., -0.0245,  0.0524,  0.0311],
         [-1.3197,  0.4543, -0.5534,  ...,  0.0156,  0.0123,  0.0180],
         [-1.0368, -0.0070, -0.3843,  ..., -0.0023,  0.0444,  0.0364]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 768])
hidden_states tensor([[[-0.0522, -0.1287, -0.0559,  ...,  0.0056, -0.0125, -0.0099],
         [ 0.0331, -0.0205, -0.1084,  ...,  0.0435, -0.0306

NameError: name 'ds' is not defined

In [51]:
# save output['hidden_states'][0][0]

output['hidden_states'][0][0].detach().numpy().tofile("gpt_weights/input.bin")
output.logits[0].detach().numpy().tofile("gpt_weights/output.bin")